In [4]:
import asyncio
import nest_asyncio
from utils import async_sheet_from_df, get_creds
import pandas as pd
from pandas import json_normalize
import pandas_gbq
import gspread_asyncio
from gspread.exceptions import APIError
import dotenv
import db_dtypes

nest_asyncio.apply()
dotenv.load_dotenv()

True

In [5]:
creds = get_creds()

In [6]:
async def get_data_from_bigquery(time_range):
    query = f"""
        SELECT *
        FROM
            `bigquery-public-data.google_analytics_sample.ga_sessions_*`
        WHERE
            date BETWEEN '{time_range[0]}' AND '{time_range[1]}'
    """
    loop = asyncio.get_running_loop()
    df = await loop.run_in_executor(None, lambda: pandas_gbq.read_gbq(query, project_id=creds.project_id, dialect="standard"))

    return df


In [7]:
async def get_results(time_ranges: list):
    results = await asyncio.gather(*[get_data_from_bigquery(time_range) for time_range in time_ranges])
    return results


In [8]:
TIME_RANGES = [
    ("20170101", "20170107"),
    ("20170201", "20170207"),
    ("20170301", "20170307")
]


In [9]:
results = asyncio.run(get_results(TIME_RANGES))


Downloading:  71%|███████   |
Downloading:  73%|███████▎  |
Downloading:  74%|███████▍  |
Downloading:  82%|████████▏ |
Downloading:  91%|█████████▏|
Downloading:  96%|█████████▌|
Downloading:  98%|█████████▊|
Downloading: 100%|██████████|
Downloading:  99%|█████████▉|
Downloading: 100%|█████████▉|
Downloading: 100%|█████████▉|
Downloading: 100%|██████████|



























Downloading: 100%|██████████|


In [66]:
visits_df = pd.concat(results, ignore_index=True)
visits_df["date"] = pd.to_datetime(visits_df["date"], format="%Y%m%d")


In [67]:
df_device_normalized = json_normalize(visits_df["device"])
df_geo_normalized = json_normalize(visits_df["geoNetwork"])
df_traffic_normalized = json_normalize(visits_df["trafficSource"])
df_totals_normalized = json_normalize(visits_df["totals"])


In [68]:
visits_df = visits_df.drop(["device", "geoNetwork", "trafficSource", "totals", "customDimensions", "hits"], axis=1)


In [69]:
visits_df = pd.concat([visits_df, df_device_normalized, df_geo_normalized, df_traffic_normalized, df_totals_normalized], axis=1)

nunique = visits_df.nunique()
cols_to_drop = nunique[nunique == 1].index
visits_df.drop(cols_to_drop, axis=1)


,visitorId,visitNumber,visitId,visitStartTime,date,fullVisitorId,userId,clientId,channelGrouping,browser,...,hits,pageviews,timeOnSite,transactions,transactionRevenue,screenviews,uniqueScreenviews,timeOnScreen,totalTransactionRevenue,sessionQualityDim
0,<NA>,2,1483290878,1483290878,2017-01-01,7431279462169656568,None,None,Organic Search,Chrome,...,2,1.0,NaN,NaN,NaN,None,None,None,NaN,None
1,<NA>,1,1483293597,1483293597,2017-01-01,1336484329946561874,None,None,Referral,Safari,...,2,2.0,12.0,NaN,NaN,None,None,None,NaN,None
2,<NA>,1,1483292307,1483292307,2017-01-01,1701623065972643878,None,None,Organic Search,Chrome,...,2,2.0,26.0,NaN,NaN,None,None,None,NaN,None
3,<NA>,1,1483299786,1483299786,2017-01-01,398831489799928961,None,None,Organic Search,Safari,...,2,2.0,21.0,NaN,NaN,None,None,None,NaN,None
4,<NA>,3,1483305691,1483305691,2017-01-01,5139184322193043543,None,None,Display,Chrome,...,2,2.0,17.0,NaN,NaN,None,None,None,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43955,<NA>,1,1488901155,1488901155,2017-03-07,361943246310638051,None,None,Organic Search,Chrome,...,106,74.0,1393.0,NaN,NaN,None,None,None,NaN,None
43956,<NA>,2,1488917517,1488917517,2017-03-07,2048511066419559359,None,None,Referral,Chrome,...,110,76.0,1034.0,1.0,319610000.0,None,None,None,328610000.0,None
43957,<NA>,344,1488914529,1488914529,2017-03-07,0824839726118485274,None,None,Organic Search,Chrome,...,114,94.0,6140.0,NaN,NaN,None,None,None,NaN,None
43958,<NA>,10,1488929967,1488929967,2017-03-07,3305485862461522985,None,None,Direct,Safari,...,133,97.0,2876.0,NaN,NaN,None,None,None,NaN,None


In [77]:
by_browser = visits_df.groupby(['browser', "operatingSystem"]).agg(
    visits_count=('visits', 'count'),
    ).sort_values(by=["browser", "visits_count"], ascending=True).reset_index()

by_date = visits_df.groupby("date").agg(
    visit_count=("visits", "count")
).sort_values(by="date").reset_index()


In [78]:
df_dict = {
    "by_browser": by_browser,
    "by_date": by_date
}


In [79]:
async def load_to_googlesheets(sheet_url: str, worksheet_name: str, dataframe: pd.DataFrame) -> None:
    agcm = gspread_asyncio.AsyncioGspreadClientManager(get_creds)
    client = await agcm.authorize()

    spreadsheet = await client.open_by_url(sheet_url)
    try:
        worksheet = await spreadsheet.add_worksheet(title=worksheet_name, rows=1, cols=1)
    except APIError:
        worksheet = await spreadsheet.worksheet(worksheet_name)
    await async_sheet_from_df(worksheet=worksheet, dataframe=dataframe)


In [ ]:
async def main():
    tasks = [
        load_to_googlesheets(
            sheet_url="https://docs.google.com/spreadsheets/d/1gkBvCLKLbYHOjaRUb_3tOvnUuOvl7BHYmBTcnfCH6Jg/edit?usp=sharing",
            worksheet_name=worksheet_name,
            dataframe=df
        )
        for worksheet_name, df in df_dict.items()
    ]
    await asyncio.gather(*tasks)


In [80]:
asyncio.run(main())

KeyboardInterrupt: 